<a href="https://colab.research.google.com/github/thomasavare/DNLP-project/blob/master/mag_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=6819102939e729fd62334ed095edd4b8393ebaa5fedd18410813069d0b1105c5
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [ ]:
!pip install awscli
!aws s3 sync --no-sign-request s3://ai2-s2-research-public/specter/scidocs/ data/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 102.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.4/662.4 KB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 13.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
download: s3://ai2-s2-research-public/specter/scidocs/mag/te

In [ ]:
import pandas as pd

train_path, test_path = '/content/data/mag/train.csv', '/content/data/mag/test.csv'
train_df, test_df = pd.read_csv(train_path), pd.read_csv(test_path)
train_df

,pid,class_label
0,78f6417635b95decc79e83d42812de50906a562c,10
1,bc636605d012c2a118e8390996843ffe9dc047b0,11
2,a093926d80986aaa93fdc609ce8b125954b87f6b,11
3,84dd81ec2622c433d26f8908d4bc1afd3bff8f6c,18
4,026a5dc5aeddd00b2323b2bedfc91b058c145934,17
...,...,...
17496,f648e027d210e68578d46501f14e581721f449aa,1
17497,78a254f0e8a03762cca5bbc0c7b0f39fc58229ca,2
17498,2eb64dd80227373760ac96fa08a46656e6516cd7,16
17499,2ecca4407aed9ecddac4578cd880e14f82600d0b,1


In [ ]:
embeddings_df = pd.read_json('/content/drive/MyDrive/DNLP-project/embeddings_metadata_mag_mesh.json', orient='index')

In [ ]:
train_merged_df, test_merged_df = pd.merge(embeddings_df, train_df, how='inner', left_on='paper_id', right_on='pid'), pd.merge(embeddings_df, test_df, how='inner', left_on='paper_id', right_on='pid')
train_merged_df, test_merged_df = train_merged_df[['paper_id', 'embedding', 'class_label']], test_merged_df[['paper_id', 'embedding', 'class_label']]

In [ ]:
import numpy as np

x_train, y_train = [], []
x_test, y_test = [], []

i = 0
for _, embedding, class_label in train_merged_df.values:
  x_train.append(embedding)
  y_train.append(class_label)

x_train = np.array(x_train)
y_train = np.array(y_train)

for _, embedding, class_label in test_merged_df.values:
  x_test.append(embedding)
  y_test.append(class_label)

x_test = np.array(x_test)
y_test = np.array(y_test)

In [ ]:
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
import pickle

Linear SVC

In [ ]:
from sklearn.svm import LinearSVC

parameters = {'C': np.logspace(-4, 2, 7)}
svc = LinearSVC(max_iter=50000)
clf1 = GridSearchCV(svc, parameters, cv=3, scoring=make_scorer(f1_score, average="macro"))

In [ ]:
clf1.fit(x_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warning

GridSearchCV(cv=3, estimator=LinearSVC(max_iter=50000),
             param_grid={'C': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])},
             scoring=make_scorer(f1_score, average=macro))

In [ ]:
y_pred = clf1.predict(x_test)
clf1.score(x_test, y_test), np.mean(y_test == y_pred)

(0.7471634790922824, 0.7592641962143428)

In [ ]:
filename = '/content/drive/MyDrive/DNLP-project/linearSVC_model_mag.sav'
pickle.dump(clf1, open(filename, 'wb'))

SVC

In [ ]:
from sklearn.svm import SVC

parameters = {'C': np.logspace(-4, 2, 7)}
svc = SVC()
clf2 = GridSearchCV(svc, parameters, cv=3, scoring=make_scorer(f1_score, average="macro"))

In [ ]:
clf2.fit(x_train, y_train)

GridSearchCV(cv=3, estimator=SVC(),
             param_grid={'C': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])},
             scoring=make_scorer(f1_score, average=macro))

In [ ]:
y_pred = clf2.predict(x_test)
clf2.score(x_test, y_test), np.mean(y_test == y_pred)

(0.7434185578434368, 0.7525993068515062)

In [ ]:
filename = '/content/drive/MyDrive/DNLP-project/SVC_model_mag.sav'
pickle.dump(clf2, open(filename, 'wb'))

Gaussian naive bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()

In [ ]:
gnb.fit(x_train, y_train)

GaussianNB()

In [ ]:
y_pred = gnb.predict(x_test)
gnb.score(x_test, y_test), f1_score(y_test, y_pred, average='macro')

(0.7094108237803253, 0.6978830891292475)

In [ ]:
filename = '/content/drive/MyDrive/DNLP-project/GNB_model_mag.sav'
pickle.dump(gnb, open(filename, 'wb'))

Knn

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knb = KNeighborsClassifier()

In [ ]:
knb.fit(x_train, y_train)

KNeighborsClassifier()

In [ ]:
y_pred = knb.predict(x_test)
knb.score(x_test, y_test), f1_score(y_test, y_pred, average="macro")

(0.7014129565449213, 0.6925303177923376)

In [ ]:
filename = '/content/drive/MyDrive/DNLP-project/Knn_model_mag.sav'
pickle.dump(knb, open(filename, 'wb'))

SGD

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier()

In [ ]:
sgd.fit(x_train, y_train)

SGDClassifier()

In [ ]:
y_pred = sgd.predict(x_test)
sgd.score(x_test, y_test), f1_score(y_test, y_pred, average="macro")

(0.728872300719808, 0.716179891386679)

In [ ]:
filename = '/content/drive/MyDrive/DNLP-project/SGD_model_mag.sav'
pickle.dump(knb, open(filename, 'wb'))